<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Tutorial on generating an explanation for an image-based binary classifier model on Watson OpenScale

## Contents:
- [1. Setup](#setup)
- [2. Creating and deploying an image-based model](#deployment)
- [3. Subscriptions](#subscription)
- [4. Explainability](#explainability)

<a id="setup"></a>
## 1. Setup

### 1.1 Install AIOS and WML packages

In [ ]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

In [ ]:
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1

Note: Restart the kernel to assure the new libraries are being used.

### 1.2 Configure credentials

Get AIOS (AI Openscale) `apikey` by going to the [Bluemix console](https://console.bluemix.net/) and clicking `Manage->Account->Users`. Select `Platform API Keys` from the sidebar and then click the "Create" button.

One can obtain the AIOS `instance_id` (guid) by accessing the [cloud console](https://console.bluemix.net/services) and clicking anywhere on the AIOS service tile except for the service link and then checking the popping sidebar on the right.

In [ ]:
AIOS_CREDENTIALS = {
    "instance_guid": "*****",
    "apikey": "*****", 
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

Generate or fetch the WML credentials by clicking on `Credentials` in the sidebar of the provisioned WML page and paste it below.

In [ ]:
WML_CREDENTIALS = {
    "apikey": "*****",
    "iam_apikey_description": "*****",
    "iam_apikey_name": "*****",
    "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
    "iam_serviceid_crn": "*****",
    "instance_id": "*****",
    "password": "*****",
    "url": "https://us-south.ml.cloud.ibm.com",
    "username": "*****"
}

<a id="deployment"></a>
## 2. Creating and deploying an image-based model

We are going to create a binary classifier which classifies an image as a Dog or a Cat (Probability: 1 = dog, 0 = cat). The dataset can be downloaded from here: https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data. The dataset can also be found here: https://ibm.box.com/shared/static/itl0el289mz06py2e6aemehx6lge1rou.zip

Now, create a folder named `data` and inside it create subdirectories: `train` and `validation`. Further, create folders named `dogs` and `cats` (as shown below) with 1024 dog and cat images in the `train` directory and 416 dog and cat images in the `validation` directory respectively. Post unzipping the downloaded zip file, use the images from the `train` folder found after unzipping `train.zip`.

```python
data/
    train/
        dogs/ # 1024 pictures
            dog.1.jpg
            dog.2.jpg
            ...
        cats/ # 1024 pictures
            cat.1.jpg
            cat.2.jpg
            ...
    validation/
        dogs/ # 416 pictures
            dog.1025.jpg
            dog.1026.jpg
            ...
        cats/ # 416 pictures
            cat.1025.jpg
            cat.1026.jpg
            ...
```

Note: Tensorflow versions supported by WML are: 1.2, 1.5, and 1.11. Make sure you have one of these versions before creating the models. Version 1.11 is used in this notebook.

In [ ]:
!pip install keras
!pip install tensorflow==1.11.0
!pip install keras_sequential_ascii
!pip install numpy
!pip install pillow

import keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
from keras_sequential_ascii import sequential_model_to_ascii_printout
from keras import backend as keras_backend
from keras import optimizers
from keras import applications
from keras.models import Model
import numpy as np
print(keras.__version__)

### 2.1 Creating a model

In [ ]:
# Dimension of the images
img_width, img_height = 90, 90

train_data_dir = 'dogs-cats/data/train'
validation_data_dir = 'dogs-cats/data/validation'

Note: Please modify the paths above accordingly.

In [ ]:
# Preprocessing

#used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
# Define Model

def base_model():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    
    return model

In [ ]:
epochs = 10 # One can increase the no. of epochs to get better accuracy
train_samples = 2048
validation_samples = 832

In [ ]:
cnn_n = base_model()
cnn_n.summary()

In [ ]:
# Vizualizing model structure
sequential_model_to_ascii_printout(cnn_n)

In [ ]:
# Train the model
cnn_n.fit_generator(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_steps=validation_samples // batch_size,
    validation_data=validation_generator
)

In [ ]:
cnn_n.save('dog_cat_cnn.h5')
!rm dog_cat_cnn.tar*
!tar -czvf dog_cat_cnn.tar.gz dog_cat_cnn.h5
!rm dog_cat_cnn.h5

In [ ]:
scores = cnn_n.evaluate_generator(validation_generator, validation_samples)
print(scores)
print("Accuracy: %.2f%%" % (scores[1]*100))

### 2.2 Storing the model

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

model_name = "Dog-Cat binary"

# Update the FRAMEWORK_VERSION below depending on the tensorflow version used
model_meta = {
    wml_client.repository.ModelMetaNames.NAME: model_name,
    wml_client.repository.ModelMetaNames.DESCRIPTION: "Dog-Cat binary",
    wml_client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
    wml_client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.11",
    wml_client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [
         {"name": "keras", "version": "2.2.4"}
    ]
}

In [ ]:
published_model_details = wml_client.repository.store_model(model='dog_cat_cnn.tar.gz', meta_props=model_meta)

In [ ]:
model_uid = wml_client.repository.get_model_uid(published_model_details)
print(model_uid)

### 2.3 Deploying the model

In [ ]:
deployment= wml_client.deployments.create(name= model_name + " Deployment", model_uid=model_uid)

In [ ]:
scoring_url = wml_client.deployments.get_scoring_url(deployment)
print(scoring_url)

In [ ]:
!pip install matplotlib
!pip install opencv-python 

import numpy as np
import matplotlib.pyplot as plt
import cv2

%matplotlib inline 
img = cv2.imread("dogs-vs-cats/data/train/dogs/dog.4.jpg")
img = cv2.resize(img, (90, 90))
print(img.shape)
plt.imshow(img, cmap='gray')
plt.show()

## 3. Subscriptions

### 3.1 Configuring AIOS

In [ ]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import WatsonMachineLearningAsset

aios_client = APIClient(AIOS_CREDENTIALS)
aios_client.version

### 3.2 Subscribe the asset

In [ ]:
from ibm_ai_openscale.supporting_classes import *

subscription = aios_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.UNSTRUCTURED_IMAGE,
    probability_column='probability'
))

### 3.3 Score the model and get transaction-id

In [ ]:
scoring_data = {'values': [img.tolist()]}
predictions = wml_client.deployments.score(scoring_url, scoring_data)
print(predictions)

In [ ]:
transaction_id = subscription.payload_logging.get_table_content().scoring_id[0]
print(transaction_id)

<a id="explainability"></a>
## 4. Explainability

### 4.1 Configure Explainability

In [ ]:
subscription.explainability.enable()

In [ ]:
subscription.explainability.get_details()

### 4.2 Get explanation for the transaction

In [ ]:
explanation = subscription.explainability.run(transaction_id, background_mode=False)

In [ ]:
explanation

### The explanation images can be obtained using the cells below

In [ ]:
!pip install Pillow
from PIL import Image
import base64
import io

pred = explanation["entity"]["predictions"][0]
print("Explanation for {} region:".format(pred["value"]))

img = pred["explanation_features"][0]["full_image"]
img_data = base64.b64decode(img)
Image.open(io.BytesIO(img_data))

In [ ]:
pred = explanation["entity"]["predictions"][1]
print("Explanation for {} region:".format(pred["value"]))

img = pred["explanation_features"][0]["full_image"]
img_data = base64.b64decode(img)
Image.open(io.BytesIO(img_data))